In [ ]:
import numpy as np

# Function to read the DELTA Energy Terms section from a text file
def read_delta_energy_section(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    # Find the start of the DELTA Energy Terms section
    start_index = None
    for i, line in enumerate(lines):
        if "DELTA Energy Terms" in line:
            start_index = i
            break
    # Extract relevant lines (EEL and EGB columns)
    data = []
    for line in lines[start_index + 1:]:
        if line.strip() == "":  # Stop at the first empty line after the section
            break
        columns = line.split(',')
        try:
            # Extract EEL and EGB values, convert them to float
            eel = float(columns[2].strip())
            egb = float(columns[3].strip())
            data.append((eel, egb))
        except ValueError:
            continue  # Skip lines that don't have valid numbers
    return data
    
# Function to calculate autocorrelation time
def calculate_autocorrelation_time(data):
    n = len(data)
    mean = np.mean(data)
    var = np.var(data)
    autocorr = np.correlate(data - mean, data - mean, mode='full')[n - 1:] / (var * n)
    # Identify the first point where autocorrelation drops below 1/e
    tau = np.argmax(autocorr < np.exp(-1))
    return tau if tau > 0 else 1  # Ensure at least 1 frame decorrelation
# Function to calculate mean, standard deviation, and standard error
def calculate_statistics(values):
    mean_val = np.mean(values)
    std_val = np.std(values)
    return mean_val, std_val
    
# List of file names
file_names = ['sim1.txt', 'sim2.txt', 'sim3.txt']

# Store the mean, standard deviation for each file
mean_values = []
std_values = []
num_frames_per_file = []

# Collect all decorrelated frames from all systems
all_decorrelated_values = []

# Loop over files and read the data
for file in file_names:
    delta_energy_data = read_delta_energy_section(file)
    # Combine the EEL and EGB values (EEL is negative and EGB is positive)
    combined_file_values = np.array([eel + egb for eel, egb in delta_energy_data])
    # Calculate the autocorrelation time
    tau = calculate_autocorrelation_time(combined_file_values)
    print(f"Autocorrelation time for {file}: {tau} frames")
    # Select decorrelated frames based on tau
    decorrelated_values = combined_file_values[::tau]
    # Append decorrelated frames to the combined list
    all_decorrelated_values.extend(decorrelated_values)
    # Store the number of frames used for this file
    num_frames_per_file.append(len(decorrelated_values))
    # Calculate statistics for decorrelated frames
    mean_val, std_val = calculate_statistics(decorrelated_values)
    # Append the statistics to the respective lists
    mean_values.append(mean_val)
    std_values.append(std_val)
    
# Convert to numpy array for easier manipulation
all_decorrelated_values = np.array(all_decorrelated_values)

# Calculate combined mean and standard error from all decorrelated values
combined_mean = np.mean(all_decorrelated_values)
combined_std = np.std(all_decorrelated_values)
combined_stderr = combined_std / np.sqrt(len(all_decorrelated_values))

# Output the results
print(f"Mean values for each system (mean of decorrelated frames): {mean_values}")
print(f"Standard deviation values for each system: {std_values}")
print(f"Number of decorrelated frames used per system: {num_frames_per_file}")
print(f"Total number of decorrelated frames used: {len(all_decorrelated_values)}")
print(f"Combined Mean from all decorrelated frames: {combined_mean}")
print(f"Combined Standard Deviation from all decorrelated frames: {combined_std}")
print(f"Combined Standard Error: {combined_stderr}")

# Save the results to a new file
with open('decorrelated_statistics.txt', 'w') as outfile:
    outfile.write(f"Mean values for each system: {mean_values}\n")
    outfile.write(f"Standard deviation values for each system: {std_values}\n")
    outfile.write(f"Number of decorrelated frames used per system: {num_frames_per_file}\n")
    outfile.write(f"Total number of decorrelated frames used: {len(all_decorrelated_values)}\n")
    outfile.write(f"Combined Mean of Decorrelated Frames: {combined_mean}\n")
    outfile.write(f"Combined Standard Deviation of Decorrelated Frames: {combined_std}\n")
    outfile.write(f"Combined Standard Error: {combined_stderr}\n")
print("Statistics calculated and saved to 'decorrelated_statistics.txt'")